In [1]:
import pandas as pd
import gmaps
import requests
import json
import numpy as np
from config import gkey

In [2]:
weather_df = pd.read_csv('../WeatherPy/Weather_data.csv')
weather_df.head()

,Unnamed: 0,lat,lng,Temp,Humidity,Cloudiness,Wind_Speed,City,Country
0,0,5.4112,100.3354,78.78,89.0,20.0,4.74,georgetown,sh
1,1,-33.0153,27.9116,68.11,68.0,100.0,9.31,east london,za
2,2,-3.7728,-61.6442,76.86,93.0,82.0,2.84,anori,br
3,3,57.7900,-152.4072,42.80,75.0,90.0,10.36,kodiak,us
4,4,-33.9258,18.4232,60.37,94.0,20.0,8.05,cape town,za


In [3]:
gmaps.configure(api_key=gkey)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(weather_df[["lat","lng"]], weather_df.Humidity)
heatmap_layer.maxintensity = weather_df.Humidity.max()
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
ideal_df = weather_df.loc[((weather_df.Temp > 50) & (weather_df.Temp < 85)) \
                          & (weather_df.Humidity < 65) \
                          & (weather_df.Cloudiness > 20), :]
ideal_df.head()

,Unnamed: 0,lat,lng,Temp,Humidity,Cloudiness,Wind_Speed,City,Country
11,11,42.6001,-73.9662,56.07,47.0,88.0,1.01,albany,au
17,17,33.1940,-82.5271,74.44,50.0,75.0,5.75,avera,pf
27,31,41.3712,-73.4140,56.61,58.0,90.0,11.50,bethel,us
32,36,-33.5906,26.8910,70.09,61.0,100.0,12.93,port alfred,za
43,54,5.3505,48.5268,83.98,64.0,56.0,10.18,hobyo,so


In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
name = []

for index, row in ideal_df.iterrows():
    params = {
        "location": f"{row['lat']}, {row['lng']}",
        "radius": 5000,
        "type": "lodging",
        "key": gkey
    }
    try:
        response = requests.get(base_url, params).json()['results'][0]
        print(response['name'])
        name.append(response['name'])
    except:
        print("Missing info")
        name.append(np.nan)

Missing info
Missing info
Courtyard by Marriott Danbury
The Halyards Hotel
Halane Hotel and Restaurent
Mantra on Collins Hobart
Casa Laguna, Bed & Breakfast
Hotel Tesoro Los Cabos
La Quinta Inn & Suites by Wyndham Buena Park
The Freeport Inn and Marina
Diego De Almagro Coyhaique
The Westport Inn
Marriott's Ko Olina Beach Club
Die Lemoenhuis
Embassy Suites by Hilton Portland Downtown
Resorts Resource Group International
Hotel Pizarro
Hospedaje Costa Bella
Chenango Valley State Park
Times Premier Lodge
Undulata Country Lodge
Don Eddie's Sport Fishing Center
Goto Guesthouse Utojuku & SA Car Rental
Wedgewood Motel
Scenic Hotel Southern Cross
Hampton Inn Elizabeth City
Hampton Inn Ukiah
Los Portales
Hotel Atlântico Rio Grande
خدمات موبایل میرزایی
Maui Seaside Hotel
Quality Inn Hackettstown - Long Valley
Staybridge Suites Philadelphia-Mt. Laurel
Palmar Hotel Casino
Bora-Bora Pearl Beach Resort & Spa
فندق راحة السلام
Pudong Holiday Hotel
NUR
Laurel Inn & Conference Center
Tieling 99 Yuejia Bu

In [6]:
ideal_df['Hotel Name'] = name

<ipython-input-6-09b3bc717088>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df['Hotel Name'] = name


In [7]:
ideal_df = ideal_df.dropna()
ideal_df["Hotel Name"] = ideal_df["Hotel Name"].astype("string")
ideal_df.head()

,Unnamed: 0,lat,lng,Temp,Humidity,Cloudiness,Wind_Speed,City,Country,Hotel Name
27,31,41.3712,-73.4140,56.61,58.0,90.0,11.50,bethel,us,Courtyard by Marriott Danbury
32,36,-33.5906,26.8910,70.09,61.0,100.0,12.93,port alfred,za,The Halyards Hotel
43,54,5.3505,48.5268,83.98,64.0,56.0,10.18,hobyo,so,Halane Hotel and Restaurent
44,55,-42.8794,147.3294,56.48,58.0,75.0,9.22,hobart,au,Mantra on Collins Hobart
57,76,27.9769,-114.0611,67.71,55.0,99.0,15.70,guerrero negro,mx,"Casa Laguna, Bed & Breakfast"


In [8]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(weather_df[["lat","lng"]], weather_df["Humidity"])
heatmap_layer.maxintensity = weather_df.Humidity.max()
fig.add_layer(heatmap_layer)

markers = gmaps.marker_layer(ideal_df[["lat","lng"]], info_box_content=hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…